In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer,
    ImageOverlay,
    Polyline,
    Polygon,
    Rectangle,
    Circle,
    CircleMarker,
    GeoJSON,
    GeomanDrawControl,
)

from traitlets import link

In [2]:
import json

with open("simple.geo.json") as f:
    data = json.load(f)

In [3]:
d1 = data.copy()
d2 = data.copy()

d1["features"] = d1["features"][:1]
d2["features"] = d2["features"][1:]


In [4]:
center = [46.475212657477016, 6.3198722284199675]
zoom = 9
m = Map(center=center, zoom=zoom, layout=dict(height="600px"))

g1 = GeoJSON(data=d1, pm_ignore=True, snap_ignore=True,     
            style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
           )

g2 = GeoJSON(data=d2, pm_ignore=True, snap_ignore=False)
m.add(g1)
m.add(g2)

draw_data = data["features"][1]
draw_data["properties"]["type"] = "circlemarker"
dc = GeomanDrawControl(
    marker={},
    circlemarker={},
    polygon={},
    data=[draw_data]
)

def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)


dc.on_draw(handle_draw)
m.add(dc)
m

Map(center=[46.475212657477016, 6.3198722284199675], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [10]:
# Modifying draw options will update the toolbar
dc.marker={"markerStyle": {"color": "#FF0000"}}
dc.rectangle={"pathOptions": {"color": "#FF0000"}}
dc.circlemarker={"pathOptions": {"color": "#FF0000"}}
m

Map(center=[46.475212657477016, 6.3198722284199675], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [ ]:
@solara.component
def EditMap(edit):
    center = [46.475212657477016, 6.3198722284199675]
    zoom = 9

    # edit, set_edit = solara.use_state(True)
    # edit = solara.reactive(True)

    g1 = GeoJSON.element(
        data=d1, pm_ignore=True, snap_ignore=True,
        style={
            'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1, 'color': '#666666', 'fillColor': '#666666'
        },
    )

    g2 = GeoJSON.element(
        data=d2, pm_ignore=True, snap_ignore=True,
        style={
            'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1, 'color': '#666666', 'fillColor': '#666666'
        }
    )

    dc = GeomanDrawControlComponent(
        marker={"shapeOptions": {"color": "#FF0000"}},
        # rectangle={"shapeOptions": {"color": "#FF0000"}},
        rectangle={},
        # circle={"shapeOptions": {"color": "#FF0000"}},
        circle={},
        circlemarker={"shapeOptions": {"color": "#FF0000"}},
        data=[data["features"][1]],
        edit=edit.value,
        on_draw=handle_draw
    )

    layers = [
        g1, g2
    ]

    controls = [dc]

    ipyleaflet.Map.element(
                layout=dict(height="400px"),
                layers=layers,
                center=center,
                zoom=zoom,
                controls=controls,
                # zoom_control=False,
                scroll_wheel_zoom=True,
            ).meta(ref="map-widget")